In [1]:
import h5py
import torch
import os
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# --- ١. ڕێکخستنەکان ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_PATH = '/kaggle/input/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final'
CHARS = ['BLANK'] + list(" abcdefghijklmnopqrstuvwxyz' ")
CHAR_MAP = {c: i for i, c in enumerate(CHARS)}

class NeuralSignalDataset(Dataset):
    def __init__(self, file_paths):
        self.samples = []
        for path in tqdm(file_paths, desc="Loading Data"):
            if not os.path.exists(path): continue
            with h5py.File(path, 'r') as hf:
                for key in hf.keys():
                    try:
                        f = hf[key]['input_features'][()]
                        f = (f - np.mean(f, axis=0)) / (np.std(f, axis=0) + 1e-6)
                        text = ""
                        for l_key in ['transcription', 'sentence']:
                            if l_key in hf[key]:
                                val = hf[key][l_key][()]
                                text = val.decode('utf-8').lower() if isinstance(val, bytes) else str(val).lower()
                                break
                        if text:
                            target = [CHAR_MAP[c] for c in text if c in CHAR_MAP]
                            if len(target) > 0: self.samples.append((f, target))
                    except: continue
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        return torch.tensor(self.samples[idx][0]).float(), torch.tensor(self.samples[idx][1]).long()

def collate_fn(batch):
    inputs, targets = zip(*batch)
    input_lens = torch.tensor([len(x) for x in inputs], dtype=torch.long)
    target_lens = torch.tensor([len(y) for y in targets], dtype=torch.long)
    inputs_padded = pad_sequence(inputs, batch_first=True)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded, input_lens, target_lens

# --- ٢. مۆدێل ---
class BrainTransformer(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=256, nhead=8, num_layers=4):
        super().__init__()
        self.proj = nn.Linear(input_dim, hidden_dim)
        self.upsample = nn.ConvTranspose1d(hidden_dim, hidden_dim, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.pos_enc = nn.Parameter(torch.randn(1, 5000, hidden_dim))
        enc_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=0.1, batch_first=True)
        self.transformer = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, len(CHARS))
    def forward(self, x):
        x = self.proj(x)
        x = x.transpose(1, 2)
        x = self.upsample(x)
        x = x.transpose(1, 2)
        x = x + self.pos_enc[:, :x.size(1), :]
        x = self.transformer(x)
        return self.fc(x)

# --- ٣. ڕاهێنان و پێشبینی ---
def start_process():
    train_files = [str(p) for p in Path(BASE_PATH).rglob('data_train.hdf5')]
    dataset = NeuralSignalDataset(train_files)
    loader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
    
    model = BrainTransformer().to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
    criterion = nn.CTCLoss(blank=0, zero_infinity=True)
    
    # لێرەدا تەنها ٢ ئایپۆک دامناوە بۆ ئەوەی زوو تەواو بێت
    for epoch in range(2):
        model.train()
        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/2")
        for batch in pbar:
            inputs, targets, in_lens, tar_lens = [b.to(DEVICE) for b in batch]
            optimizer.zero_grad()
            logits = model(inputs)
            log_probs = logits.permute(1, 0, 2).log_softmax(2)
            loss = criterion(log_probs, targets, in_lens * 2, tar_lens)
            loss.backward()
            optimizer.step()
            pbar.set_postfix(loss=loss.item())

    # یەکسەر دوای ڕاهێنان فایلی Submission دروست دەکات
    model.eval()
    results = []
    test_files = sorted(list(Path(BASE_PATH).rglob("data_test.hdf5")))
    with torch.no_grad():
        for f_path in tqdm(test_files, desc="Inference"):
            with h5py.File(f_path, "r") as hf:
                keys = sorted(hf.keys(), key=lambda k: int(k.split('_')[1]) if '_' in k else 0)
                for key in keys:
                    x = torch.from_numpy(hf[key]["input_features"][()]).float().unsqueeze(0).to(DEVICE)
                    x = (x - x.mean()) / (x.std() + 1e-6)
                    logits = model(x)
                    indices = torch.argmax(logits[0], dim=-1).unique_consecutive()
                    decoded = "".join([CHARS[i] for i in indices if i != 0]).strip()
                    results.append(decoded if decoded else "silence")
    
    pd.DataFrame({"id": range(len(results)), "text": results}).to_csv("submission.csv", index=False)
    print("🏁 تەواو! فایلی submission.csv دروست بوو.")

if __name__ == "__main__":
    start_process()

Loading Data:   0%|          | 0/45 [00:00<?, ?it/s]

Epoch 1/2:   0%|          | 0/505 [00:00<?, ?it/s]

Epoch 2/2:   0%|          | 0/505 [00:00<?, ?it/s]

Inference:   0%|          | 0/41 [00:00<?, ?it/s]

🏁 تەواو! فایلی submission.csv دروست بوو.


Approach Summary: Brain-to-Text Transformer with CTC Loss

1. Introduction
This approach aims to decode neural signals into text using a deep learning architecture specifically designed for sequence-to-sequence mapping. The model processes high-dimensional neural features and converts them into English characters, enabling communication directly from brain activity.

2. Data Preprocessing
Normalization: Input features are normalized using Z-score normalization (mean subtraction and division by standard deviation) to ensure stable gradient flow during training.
Target Encoding: Text transcriptions are mapped to a character-based vocabulary (a-z, spaces, and punctuation) including a 'BLANK' token for CTC compatibility.
Padding: Dynamic padding is applied to both neural sequences and target text to handle variable-length inputs within batches.
3. Model Architecture
The architecture, named BrainTransformer, consists of three main components:
Linear Projection: Reduces the initial 512-dimensional neural input to a 256-dimensional latent space.
Temporal Upsampling: A ConvTranspose1d layer is utilized to increase the temporal resolution of the signal. This is crucial for CTC loss to ensure the output sequence is long enough to accommodate the predicted text.
Transformer Encoder: A 4-layer Transformer Encoder with Multi-Head Attention (8 heads) is the core engine. It captures long-range temporal dependencies within the neural signals.
CTC Output Layer: A final fully connected layer maps the features to character probabilities.
4. Training Strategy
Loss Function: Connectionist Temporal Classification (CTC Loss) is used, which allows the model to align the neural signal with the text without requiring time-aligned labels.
Optimizer: AdamW optimizer with a learning rate of 2e−4 for better weight decay and generalization.
Hardware: Training is performed on NVIDIA GPUs using PyTorch to ensure high-speed computation.
5. Inference & Decoding
During inference, the model uses Greedy Decoding. It takes the most probable character at each time step and collapses consecutive duplicates (and removes blanks) to produce the final sentence.